* [Main Page](https://oneadder.github.io/lingtypology/)
* [Glottolog](https://oneadder.github.io/lingtypology/glottolog)
* [DB APIs](https://oneadder.github.io/lingtypology/db_apis)

# Maps <a name="up"></a>

It is possible to draw interactive linguistic maps using `lingtypology`

In [2]:
import lingtypology

## 0. Table of Contents
### 1. [Maps](#maps)
#### 1.1. [Simplest example](#simplest_example)  
#### 1.2. [Features](#features)
#### 1.3. [Map Customization](#map_cust)
#### 1.4. [Accessing Glottolog](#access_glot)
#### 1.5. [Customizing Features and Controls](#feat_cont)
#### 1.6. [Stroke Features](#s_features)
#### 1.7. [Heatmaps](#heatmaps)
#### 1.8. [Lines](#lines)
#### 1.9. [Customizing markers](#markers)
#### 1.10. [Start Location/Zoom shortcuts](#shortcuts)
#### 1.11 [Color Gradients](#gradients)
#### 1.12 [Minicharts](#minicharts)
### 2. [Real examples](#real_examples)
#### 2.1. [Circassian Example](#example1)
#### 2.2. [Another way](#example2)
#### 2.3. [Consonants Example](#example3)


## 1. Maps <a name="maps"></a>
In this paragraph I will consider some basic features that you can use.

### 1.1. Simplest example <a name="simplest_example"></a>

The simplest script that draws a map with Romanian and Ukrainian languages will be:

In [3]:
m = lingtypology.LingMap(('Romanian', 'Ukrainian'))
m.create_map()

Be advised that `create_map()` returns a `Folium` map. In this notebook it is used because the map may
be displayed this way.  
To save your map as html run: `m.save('map.html')`  
To get html as `str` run: `m.render()`

### 1.2. Features <a name="features"></a>
Simply drawing languages on a map is not very interesting. Let's say that we want to mark some morphological features as different colors.

In [4]:
languages = ["Adyghe", "Kabardian", "Polish", "Russian", "Bulgarian"]
features = ["Agglutinative", "Agglutinative", "Inflected", "Inflected", "Analythic"]
m = lingtypology.LingMap(languages)
m.add_features(features)
m.create_map()

### 1.3. Customizing the map <a name="map_cust"></a>
As you can see, the map is not centered properly. It can be fixed with passing coordinates to the `start_location` attribute. You can change start zoom as well `start_zoom`.
Also, you can change colors using the `colors` attribute.

In [5]:
m.start_location = (40, 40)
m.start_zoom = 3
m.colors = ("yellowgreen", "red", "blue")
m.create_map()

### 1.4. Accessing Glottolog <a name="access_glot"></a>

You can also use `lingtypology.glottolog` to get some data from it. For example, I want to add language affiliations to popups.  
See more details [here](https://oneadder.github.io/lingtypology/glottolog).

In [6]:
affs = lingtypology.glottolog.get_affiliations(languages)
m.add_popups(affs)
m.create_map()

### 1.5. Customizing features and controls <a name="feat_cont"></a>

You can pass additional parameters to the `add_features` method.  
If for some reason you do not wish to use colors, you could use shapes.  
If you want to add controls, you can do it as well.

In [7]:
m.add_features(features, use_shapes=True, control=True)
m.create_map()

### 1.6. Stroke features <a name="s_features"></a>

It is possible to add another set of features to strokes of markers.

In [8]:
stroke_features = ['Ergative', 'Ergative', 'Accusative', 'Accusative', 'Accusative']
m = lingtypology.LingMap(languages)
m.add_features(features)
m.add_stroke_features(stroke_features)
m.create_map()

### 1.7. Heatmaps <a name="heatmaps"></a>
It would seem unfair if we could only draw circles.  
Let's draw a heatmap (the more density, the more languages with Large consonant inventory).  
It is from Wals. For more information consult [this](https://oneadder.github.io/lingtypology/db_apis).

In [9]:
from lingtypology.db_apis import Wals
wals_page = Wals('1a').get_df()

#First initialize LingMap without languages
m = lingtypology.LingMap()
#Add heatmap from  the Wals data
m.add_heatmap(wals_page[wals_page._1A == 'Large'].coordinates)
#Let's also add a title to the map
m.title = 'Large Consonant Inventories'
m.create_map()

Citation for feature 1A:
Ian Maddieson. 2013. Consonant Inventories.
In: Dryer, Matthew S. & Haspelmath, Martin (eds.)
The World Atlas of Language Structures Online.
Leipzig: Max Planck Institute for Evolutionary Anthropology.
(Available online at http://wals.info/chapter/1, Accessed on 2019-05-10.)



### 1.8. Lines <a name="lines"></a>

Let's say we have this list of languages:

In [10]:
balkan = ['Modern Greek', 'Romanian', 'Bulgarian', 'Macedonian', 'Gheg Albanian']
other = ['Ukrainian', 'Turkish', 'Italian']
languages_all = balkan + other

We want the ones from Balkan sprachbund connected with lines.

In [11]:
#Let's get the coordinates that we need to draw a line
coordinates = map(lingtypology.glottolog.get_coordinates, balkan)
m = lingtypology.LingMap(languages_all)
m.add_line(coordinates)
m.start_location = (43, 27)
m.start_zoom = 5
m.create_map()

### 1.9. Customizing markers <a name="markers"></a>
It is possible to set opacity and width of markers.

In [12]:
m = lingtypology.LingMap(languages)
m.add_features(features, radius=7, opacity=0.9)
m.add_stroke_features(stroke_features, opacity=0.7)
m.create_map()

### 1.10. Start Location/Zoom shortcuts <a name="shortcuts"></a>
There are some shortcuts that can allow not to choose `start_location` and `start_zoom` manually.

In [13]:
m = lingtypology.LingMap([
    'Irish', 'Estonian', 'Norwegian', 'Romanian', 'Polish',
    'Italian', 'Turkish', 'French', 'English', 'Spanish'
])
m.start_location = 'Central Europe'
m.create_map()

Full list of shortcuts:

In [14]:
m.start_location_mapping

{'Central Europe': {'start_location': (50, 0), 'start_zoom': 4},
 'Caucasus': {'start_location': (43, 42), 'start_zoom': 6},
 'Australia & Oceania': {'start_location': (-16, 159), 'start_zoom': 3},
 'Papua New Guinea': {'start_location': (-5, 141), 'start_zoom': 6},
 'Africa': {'start_location': (3, 22), 'start_zoom': 3},
 'Asia': {'start_location': (36, 100), 'start_zoom': 3},
 'North America': {'start_location': (51, -102), 'start_zoom': 3},
 'Central America': {'start_location': (19, -81), 'start_zoom': 4},
 'South America': {'start_location': (-27, -49), 'start_zoom': 3}}

### 1.11. Color gradients <a name="gradients"></a>
There are two ways to use color gradients.
Let's say we have features called "Millions of native speakers".

In [15]:
native_speakers = [0, 1, 5, 24, 44, 62, 78, 78, 378, 442]
m.legend_title = 'Native speakers (mln)'
m.legend_position = 'topright'

The first way is simply pass the features as numeric:

In [16]:
m.add_features(native_speakers, numeric=True)
m.colormap_colors = ('white', 'red')
m.create_map()

Or you can simply set attribute `color` to a gradient using `gradient` function.

In [17]:
m.add_features(native_speakers)
m.colors = lingtypology.gradient(10, color2='red')
m.create_map()

### 1.12. Minicharts <a name="minicharts"></a>
You can add minicharts to the map using `add_minicharts` method.

In [18]:
import pandas
data = pandas.read_csv(
    'https://raw.githubusercontent.com/ropensci/lingtypology/master/database_creation/ejective_and_n_consonants.csv',
    delimiter=',',
    header=0,
)

In [19]:
m = lingtypology.LingMap(data.language)
m.add_minicharts(data.consonants, data.vowels, names=('consonants', 'vowels'))
m.create_map()

It is possible to customize them using `colors`, `size`, `startangle` and `labels` parameters.

In [20]:
m = lingtypology.LingMap(data.language)
m.add_minicharts(
    data.consonants, data.vowels,
    names = ('consonants', 'vowels'),
    colors = ('#4363d8', '#d8b843'), #default: default colors of LingMap
    size = 0.8, #default: 0.6
    startangle = 45, #default: 90
    labels = True #default: False
)
m.create_map()

You can also use **bar** charts.

In [21]:
m = lingtypology.LingMap(data.language)
m.add_minicharts(
    data.consonants, data.vowels,
    names = ('consonants', 'vowels'),
    typ = 'bar',
    colors = ('#4363d8', '#d8b843'),
    size = 0.4
)
m.create_map()

## 2. Some real examples <a name="real_examples"></a>

### 2.1. Circassian Example <a name="example1"></a>
Let's draw a map based on data from [this](https://github.com/ropensci/lingtypology/blob/master/database_creation/circassian.csv) CSV.  

In [22]:
import pandas

In [23]:
circassian = pandas.read_csv(
    'https://raw.githubusercontent.com/ropensci/lingtypology/master/database_creation/circassian.csv',
    delimiter=',',
    header=0,
)
circassian.head()

,latitude,longitude,village,district,dialect,language
0,45.183330,39.666670,Khatukay,ra,Temirgoy,Adyghe
1,43.092518,43.522085,Psygansu,kbr,Baksan,Kabardian
2,43.217748,43.738020,Verkhny Lesken,kbr,Baksan,Kabardian
3,43.260246,43.653788,Zhemtala,kbr,Baksan,Kabardian
4,43.317842,43.876519,Erokko,kbr,Baksan,Kabardian


In [24]:
coordinates = zip(list(circassian.latitude), list(circassian.longitude))
dialects = circassian.dialect
languages = circassian.language
popups = circassian.village

In [25]:
#Creating LingMap object
m = lingtypology.LingMap(languages)
#Setting up start location
m.start_location = (44.21, 42.32)
#Setting up start zoom
m.start_zoom = 7
#Inner features < dialect
m.add_features(dialects, opacity=0.7)
#Outer features < language
m.add_stroke_features(languages, opacity=0.8)
#Popups < village
m.add_popups(popups)
#Tooltips < language
m.add_tooltips(languages)
#Custom coordinates (override the ones from Glottolog)
m.add_custom_coordinates(coordinates)
#Inner legend title and position
m.legend_title = 'Dialects'
m.legend_position = 'bottomright'
#Outer legend title and position
m.stroke_legend_title = 'Languages'
m.stroke_legend_position = 'topright'
m.create_map()

### 2.2. Another way <a name="example2"></a>
If you are used to the R package, you may find this way to program the second example more appealing.  
However, it is not recommended because most of customization is impossible.

In [26]:
coordinates = zip(list(circassian.latitude), list(circassian.longitude))
dialects = circassian.dialect
languages = circassian.language
popups = circassian.village

lingtypology.map_feature(
    languages,
    custom_coordinates = coordinates,
    features = dialects,
    stroke_features = languages,
    popups = popups,
    tooltips = languages,
    legend_title = 'Dialects',
    legend_position = 'bottomright',
    stroke_legend_title = 'Languages',
    stroke_legend_position = 'topright',
    start_zoom = 7,
    start_location = (44.21, 42.32),
    save_html = 'circassian.html',
)

### 2.3. Consonants Example <a name="example3"></a>
Map based on [this](https://raw.githubusercontent.com/ropensci/lingtypology/master/database_creation/ejective_and_n_consonants.csv) data.

In [27]:
data = pandas.read_csv(
    'https://raw.githubusercontent.com/ropensci/lingtypology/master/database_creation/ejective_and_n_consonants.csv',
    delimiter=',',
    header=0,
)

In [28]:
m = lingtypology.LingMap(data.language)
m.legend_title = 'Amount of consonants'
m.add_tooltips(data.consonants)
#If numeric is True, it will look like this
m.add_features(data.consonants, numeric=True)
m.create_map()

[Go back up](#up)